In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
# 销毁当前TF图并创建一个新图,有助于避免模型/网络层混乱
tf.keras.backend.clear_session() 

In [3]:
inputs = keras.Input(shape=(784,))

In [4]:
inputs.shape, inputs.dtype

(TensorShape([None, 784]), tf.float32)

In [5]:
dense = keras.layers.Dense(units=64, activation='relu')
x = dense(inputs)

In [6]:
x

<tf.Tensor 'dense/Identity:0' shape=(None, 64) dtype=float32>

In [7]:
x = keras.layers.Dense(units=64, activation='relu')(x)
outputs = keras.layers.Dense(units=10)(x)

In [8]:
model = keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')
model

In [9]:
model.summary()

Model: "mnist_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [10]:
keras.utils.plot_model(model, 'mnist_model.png', show_shapes=True)

InvocationException: GraphViz's executables not found

In [11]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [12]:
x_train = x_train.reshape(-1, 784).astype('float32') / 255
x_test = x_test.reshape(-1, 784).astype('float32') / 255

In [13]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             optimizer=keras.optimizers.RMSprop(),
             metrics=['accuracy'])

In [14]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_split=0.2)

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 1s 30us/sample - loss: 0.3340 - accuracy: 0.9045 - val_loss: 0.1822 - val_accuracy: 0.9466
Epoch 2/5
48000/48000 [==============================] - 1s 26us/sample - loss: 0.1567 - accuracy: 0.9535 - val_loss: 0.1424 - val_accuracy: 0.9578
Epoch 3/5
48000/48000 [==============================] - 1s 26us/sample - loss: 0.1144 - accuracy: 0.9655 - val_loss: 0.1367 - val_accuracy: 0.9597
Epoch 4/5
48000/48000 [==============================] - 1s 24us/sample - loss: 0.0919 - accuracy: 0.9721 - val_loss: 0.1235 - val_accuracy: 0.9636
Epoch 5/5
48000/48000 [==============================] - 1s 22us/sample - loss: 0.0786 - accuracy: 0.9758 - val_loss: 0.1135 - val_accuracy: 0.9672


In [15]:
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

10000/1 - 0s - loss: 0.0504 - accuracy: 0.9722
Test loss: 0.10037062031840906
Test accuracy: 0.9722


In [17]:
model.save('./logs/mymodel.h5')

In [20]:
# 先删除模型再通过load_model加载
del model
model = keras.models.load_model('./logs/mymodel.h5')

In [21]:
# encoder model: turn image inputs into 16-dimensional vetors
encoder_input = keras.Input(shape=(28, 28, 1), name='img')
x = keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu')(encoder_input)
x = keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=3)(x)
x = keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu')(x)
encoder_output = keras.layers.GlobalMaxPooling2D()(x)

encoder = keras.Model(inputs=encoder_input, outputs=encoder_output, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                0   

In [22]:
# autoencoder
x = keras.layers.Reshape((4, 4 , 1))(encoder_output)
x = keras.layers.Conv2DTranspose(filters=16, kernel_size=3, activation='relu')(x)
x = keras.layers.Conv2DTranspose(filters=32, kernel_size=3, activation='relu')(x)
x = keras.layers.UpSampling2D(size=3)(x)
x = keras.layers.Conv2DTranspose(filters=16, kernel_size=3, activation='relu')(x)
decoder_output = keras.layers.Conv2DTranspose(filters=1, kernel_size=3, activation='relu')(x)

autoencoder = keras.Model(inputs=encoder_input, outputs=decoder_output, name='autoencoder')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                